### Consumer Finances Protection Bureau (CFPB) 
#### Consumer Complaint Database

A revision of work from MIDS Capstone project (May 2016)
plus new work on this dataset

August 2017

In [190]:
import pandas as pd
import numpy as np
from datetime import *
import csv
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import time
beg_time = time.clock()

In [191]:
# Display dataframes in a spreadsheet-like format
from IPython.display import display, HTML
pd.set_option('display.notebook_repr_html',True)
pd.set_option("display.max_rows",100) 
pd.set_option('display.max_columns', None)
pd.set_option("display.html.table_schema", True)
pd.options.display.float_format = '{:20,.4f}'.format # Two decimals for floats
pd.set_option('display.colheader_justify', 'left')
pd.set_option('display.width', 180)

In [192]:
#url = "https://data.consumerfinance.gov/api/views/s6ew-h6mp/rows.csv?accessType=DOWNLOAD"
#Comp = pd.read_csv(url)

In [193]:
# Save original dataset into a CSV file so that we don't have to read from the web 
# each time we run the program
#Comp.to_csv("complaints.csv", index = False)

start_time = time.clock()
Comp = pd.read_csv("complaints.csv")
print time.clock() - start_time, "seconds"

6.4144 seconds


In [194]:
Comp.info()
#Comp.dtypes
#Comp.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830396 entries, 0 to 830395
Data columns (total 18 columns):
Date received                   830396 non-null object
Product                         830396 non-null object
Sub-product                     595245 non-null object
Issue                           830396 non-null object
Sub-issue                       364725 non-null object
Consumer complaint narrative    174473 non-null object
Company public response         224452 non-null object
Company                         830396 non-null object
State                           821051 non-null object
ZIP code                        821013 non-null object
Tags                            116337 non-null object
Consumer consent provided?      323333 non-null object
Submitted via                   830396 non-null object
Date sent to company            830396 non-null object
Company response to consumer    830396 non-null object
Timely response?                830396 non-null object
Consumer 

In [195]:
# Numeric features: only Complaint ID
# All other features are textual, categorical or date

# Here is how the dataset looks like
display(Comp)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,M&T BANK CORPORATION,MI,48382,NaN,NaN,Referral,03/17/2014,Closed with explanation,Yes,No,759217
1,10/01/2016,Credit reporting,NaN,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,NaN,Consent provided,Web,10/05/2016,Closed with explanation,Yes,No,2141773
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,NaN,I purchased a new car on XXXX XXXX. The car de...,NaN,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No,2163100
3,06/08/2014,Credit card,NaN,Bankruptcy,NaN,NaN,NaN,AMERICAN EXPRESS COMPANY,ID,83854,Older American,NaN,Web,06/10/2014,Closed with explanation,Yes,Yes,885638
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,NaN,NaN,"CITIBANK, N.A.",VA,23233,NaN,NaN,Web,09/13/2014,Closed with explanation,Yes,Yes,1027760
5,11/13/2013,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",NaN,NaN,NaN,U.S. BANCORP,MN,48322,NaN,NaN,Phone,11/20/2013,Closed with monetary relief,Yes,No,596562
6,06/16/2015,Debt collection,Medical,Improper contact or sharing of info,Contacted employer after asked not to,NaN,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,NaN,Consent not provided,Web,06/19/2015,Closed with explanation,Yes,No,1422680
7,06/15/2015,Credit reporting,NaN,Credit reporting company's investigation,Inadequate help over the phone,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,NaN,Consent provided,Web,06/15/2015,Closed with explanation,Yes,No,1420702
8,11/13/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,NaN,NaN,Referral,12/10/2015,Closed with explanation,Yes,Yes,1654890
9,10/21/2014,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,OCWEN LOAN SERVICING LLC,FL,32714,Older American,NaN,Web,10/21/2014,Closed with explanation,Yes,No,1079567


In [196]:
# Duplicate IDs: none
DupID = Comp.duplicated(subset='Complaint ID', keep=False)
print DupID.value_counts()

False    830396
dtype: int64


In [197]:
# Drop 'Complaint ID' as it will not be used in analysis
Comp.drop(['Complaint ID'], inplace = True, axis = 1)

$ $
#### Missing values
$ $

There are many missing values in the dataset. Convert those into a category: this makes working with them easier (GroupBy, etc)

$ $

In [198]:
Comp.fillna('xxxxx', inplace = True)
display(Comp)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?
0,03/12/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,03/17/2014,Closed with explanation,Yes,No
1,10/01/2016,Credit reporting,xxxxx,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,10/05/2016,Closed with explanation,Yes,No
2,10/17/2016,Consumer Loan,Vehicle loan,Managing the loan or lease,xxxxx,I purchased a new car on XXXX XXXX. The car de...,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,10/20/2016,Closed with explanation,Yes,No
3,06/08/2014,Credit card,xxxxx,Bankruptcy,xxxxx,xxxxx,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,06/10/2014,Closed with explanation,Yes,Yes
4,09/13/2014,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,xxxxx,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,09/13/2014,Closed with explanation,Yes,Yes
5,11/13/2013,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,11/20/2013,Closed with monetary relief,Yes,No
6,06/16/2015,Debt collection,Medical,Improper contact or sharing of info,Contacted employer after asked not to,xxxxx,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,06/19/2015,Closed with explanation,Yes,No
7,06/15/2015,Credit reporting,xxxxx,Credit reporting company's investigation,Inadequate help over the phone,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,06/15/2015,Closed with explanation,Yes,No
8,11/13/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,12/10/2015,Closed with explanation,Yes,Yes
9,10/21/2014,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,10/21/2014,Closed with explanation,Yes,No


$ $
#### Dates: convert from string to date
$ $

In [199]:
start_time = time.clock()
# Convert the two date features from string to date
print Comp['Date received'].dtype, Comp['Date sent to company'].dtype
Comp['DTRec'] = pd.to_datetime(Comp['Date received']) 
Comp['DTSent'] = pd.to_datetime(Comp['Date sent to company'])
print time.clock() - start_time, "seconds"

object object
291.604056 seconds


In [200]:
# Drop the initial, string-formatted date
Comp.drop(['Date received','Date sent to company'], inplace = True, axis = 1)

In [201]:
# Complaint counts by ascending date
Comp['DTRec'].value_counts() \
             .reset_index() \
             .sort_values('index', ascending = True)

,index,DTRec
1801,2011-12-01,120
1739,2011-12-02,138
2058,2011-12-03,26
2066,2011-12-04,22
1638,2011-12-05,164
1617,2011-12-06,170
1852,2011-12-07,100
1631,2011-12-08,166
1935,2011-12-09,68
2026,2011-12-10,36


$ $
#### Product and Sub-product
$ $

In [202]:
Comp['Product'].value_counts(dropna = False) \
               .reset_index() \
               .sort_values('Product',ascending=False)

,index,Product
0,Mortgage,234609
1,Debt collection,158777
2,Credit reporting,140418
3,Credit card,89187
4,Bank account or service,86206
5,Student loan,35654
6,Consumer Loan,31607
7,"Credit reporting, credit repair services, or o...",23771
8,Credit card or prepaid card,5884
9,Payday loan,5546


In [203]:
Comp['Sub-product'].value_counts(dropna = False) \
                   .reset_index()  \
                   .sort_values('Sub-product', ascending=False)

,index,Sub-product
0,xxxxx,235151
1,Other mortgage,86627
2,Conventional fixed mortgage,70613
3,Checking account,62802
4,"Other (i.e. phone, health club, etc.)",44553
5,I do not know,32284
6,Credit card,28704
7,FHA mortgage,25659
8,Conventional adjustable mortgage (ARM),25381
9,Non-federal student loan,25162


In [204]:
t1 = Comp.groupby([Comp['Product'], Comp['Sub-product']]) \
         .size() \
         .reset_index(name="Count") \
         .sort_values('Count', ascending=False)
display(t1)

,Product,Sub-product,Count
23,Credit reporting,xxxxx,140418
16,Credit card,xxxxx,89187
62,Mortgage,Other mortgage,86627
57,Mortgage,Conventional fixed mortgage,70613
2,Bank account or service,Checking account,59045
39,Debt collection,"Other (i.e. phone, health club, etc.)",44553
33,Debt collection,I do not know,32284
29,Debt collection,Credit card,28704
59,Mortgage,FHA mortgage,25659
56,Mortgage,Conventional adjustable mortgage (ARM),25381


$ $

The product - sub-product table above is copied and pasted into Google Sheets first. A new, combined product classification is created. Any subcategory that has more than 10,000 observations is kept as a separate product. Smaller sub-categories are folded into a generic "other" category.

$ $


In [205]:
Prod = pd.read_csv("Product_and_Sub_product.csv")
display(Prod)

,Product,Sub-product,Count,Prod
0,Bank account or service,Checking account,59045,Bank account or service - checking account
1,Bank account or service,Other bank product/service,17825,Bank account or service - other
2,Bank account or service,Savings account,5294,Bank account or service - other
3,Bank account or service,(CD) Certificate of deposit,3417,Bank account or service - other
4,Bank account or service,Cashing a check without an account,625,Bank account or service - other
5,Checking or savings account,Checking account,3757,Bank account or service - other
6,Checking or savings account,Other banking product or service,786,Bank account or service - other
7,Checking or savings account,Savings account,291,Bank account or service - other
8,Checking or savings account,CD (Certificate of Deposit),139,Bank account or service - other
9,Checking or savings account,Personal line of credit,2,Bank account or service - other


In [206]:
Comp = pd.merge(Comp, Prod, how='left', on=['Product', 'Sub-product'])
display(Comp)

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Count,Prod
0,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,86627,Mortgage - other
1,Credit reporting,xxxxx,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,140418,Credit reporting
2,Consumer Loan,Vehicle loan,Managing the loan or lease,xxxxx,I purchased a new car on XXXX XXXX. The car de...,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,17780,Consumer loan - vehicle loan
3,Credit card,xxxxx,Bankruptcy,xxxxx,xxxxx,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,89187,Credit card or prepaid card
4,Debt collection,Credit card,Communication tactics,Frequent or repeated calls,xxxxx,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,28704,Debt collection - credit card
5,Mortgage,Conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,25381,Mortgage - conventional adjustable mortgage (ARM)
6,Debt collection,Medical,Improper contact or sharing of info,Contacted employer after asked not to,xxxxx,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,21209,Debt collection - medical
7,Credit reporting,xxxxx,Credit reporting company's investigation,Inadequate help over the phone,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,140418,Credit reporting
8,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,86627,Mortgage - other
9,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,70613,Mortgage - conventional fixed mortgage



$ $
#### Recode Product and Sub-product
$ $


 
 
By combining Product and Sub-product, we get a new categorical feature with about 20 categories. All but four of these categories have more than 10,000 observations.

$ $


In [207]:
Comp['Prod'].value_counts(dropna = False) 

Credit reporting                                     164189
Debt collection - other                              108864
Mortgage - other                                     101330
Credit card or prepaid card                           98888
Mortgage - conventional fixed mortgage                70613
Bank account or service - checking account            59045
Bank account or service - other                       32136
Debt collection - credit card                         28704
Mortgage - FHA mortgage                               25659
Mortgage - conventional adjustable mortgage (ARM)     25381
Student loan - non-federal student loan               22279
Debt collection - medical                             21209
Consumer loan - vehicle loan                          17780
Consumer loan - other                                 13827
Student loan - federal student loan servicing         12083
Mortgage - home equity loan or line of credit         11626
Payday loan, title loan, or personal loa

In [208]:
# Drop Product, Sub-product and Count (comes from the merge)
Comp.drop(['Product','Sub-product', 'Count'], inplace = True, axis = 1)
display(Comp)

,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod
0,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other
1,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting
2,Managing the loan or lease,xxxxx,I purchased a new car on XXXX XXXX. The car de...,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan
3,Bankruptcy,xxxxx,xxxxx,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card
4,Communication tactics,Frequent or repeated calls,xxxxx,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card
5,"Loan servicing, payments, escrow account",xxxxx,xxxxx,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM)
6,Improper contact or sharing of info,Contacted employer after asked not to,xxxxx,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical
7,Credit reporting company's investigation,Inadequate help over the phone,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting
8,"Loan modification,collection,foreclosure",xxxxx,xxxxx,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other
9,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage


$ $

#### Issue and Sub-issue

$ $

These two features are combined into one in a manner similar to Product and Sub-product. there are many more categories here, though.
$ $


In [209]:
t1 = Comp.groupby([Comp['Issue'], Comp['Sub-issue']]) \
         .size() \
         .reset_index(name="Count") \
         .sort_values('Count', ascending=False)
            
t1.to_csv("issues.csv", index = False, quoting=csv.QUOTE_NONNUMERIC)            
display(t1)

,Issue,Sub-issue,Count
184,"Loan modification,collection,foreclosure",xxxxx,112313
186,"Loan servicing, payments, escrow account",xxxxx,77334
1,"Account opening, closing, or management",xxxxx,37962
162,Incorrect information on credit report,Account status,37057
57,Cont'd attempts collect debt not owed,Debt is not mine,36752
164,Incorrect information on credit report,Information is not mine,32375
99,Deposits and withdrawals,xxxxx,22850
101,Disclosure verification of debt,Not given enough info to verify debt,21816
11,"Application, originator, mortgage broker",xxxxx,17229
60,Cont'd attempts collect debt not owed,Debt was paid,16631


In [210]:
Iss = pd.read_csv("issues_recoded.csv")
display(Iss)

,Issue,Sub-issue,Count,Iss
0,"Account opening, closing, or management",xxxxx,37962,"Account opening, closing, or management"
1,Account terms and changes,xxxxx,484,Other
2,Adding money,xxxxx,202,Other
3,Advertising,Confusing or misleading advertising about the ...,7,Other
4,Advertising,Changes in terms from what was offered or adve...,2,Other
5,Advertising and marketing,xxxxx,2959,Advertising and marketing
6,"Advertising and marketing, including promotion...",Didn't receive advertised or promotional terms,235,Advertising and marketing
7,"Advertising and marketing, including promotion...",Confusing or misleading advertising about the ...,181,Advertising and marketing
8,"Advertising, marketing or disclosures",xxxxx,77,Advertising and marketing
9,Application processing delay,xxxxx,540,Other


In [211]:
Comp = pd.merge(Comp, Iss, how='left', on=['Issue', 'Sub-issue'])
display(Comp)

,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Count,Iss
0,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,112313,"Loan modification,collection,foreclosure"
1,Incorrect information on credit report,Account status,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,37057,Incorrect information on credit report - Accou...
2,Managing the loan or lease,xxxxx,I purchased a new car on XXXX XXXX. The car de...,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,15279,Managing the loan or lease
3,Bankruptcy,xxxxx,xxxxx,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,448,Other
4,Communication tactics,Frequent or repeated calls,xxxxx,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,15229,Communication tactics - Frequent or repeated c...
5,"Loan servicing, payments, escrow account",xxxxx,xxxxx,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),77334,"Loan servicing, payments, escrow account"
6,Improper contact or sharing of info,Contacted employer after asked not to,xxxxx,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,2206,Improper contact or sharing of info - Other
7,Credit reporting company's investigation,Inadequate help over the phone,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,1777,Credit reporting company's investigation - Other
8,"Loan modification,collection,foreclosure",xxxxx,xxxxx,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,112313,"Loan modification,collection,foreclosure"
9,"Loan modification,collection,foreclosure",xxxxx,xxxxx,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,112313,"Loan modification,collection,foreclosure"


In [212]:
Comp['Iss'].value_counts(dropna = False) 

Loan modification,collection,foreclosure                                               112313
Other                                                                                   82740
Loan servicing, payments, escrow account                                                77334
Account opening, closing, or management                                                 37962
Incorrect information on credit report - Account status                                 37057
Cont'd attempts collect debt not owed - Debt is not mine                                36752
Incorrect information on credit report - Information is not mine                        32375
Deposits and withdrawals                                                                22850
Disclosure verification of debt - Not given enough info to verify debt                  21816
Application, originator, mortgage broker                                                17229
Cont'd attempts collect debt not owed - Debt was paid       

In [213]:
# Drop Product, Sub-product and Count (comes from the merge)
Comp.drop(['Issue','Sub-issue', 'Count'], inplace = True, axis = 1)
display(Comp)

,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss
0,xxxxx,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure"
1,I have outdated information on my credit repor...,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...
2,I purchased a new car on XXXX XXXX. The car de...,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease
3,xxxxx,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other
4,xxxxx,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...
5,xxxxx,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account"
6,xxxxx,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other
7,An account on my credit report has a mistaken ...,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other
8,xxxxx,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure"
9,xxxxx,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure"


### Consumer complaint narrative
$ $
This section largely draws from my previous work. It cleans the narrative by stripping stop words, punctuation marks etc first. Then it counts the positive and negative words using the Loughran - McDonald finance sentiment dictionary.

$ $


$ $
Load the Loughran-McDonald dictionary of positive and negative words. Then, compute three numbers for sentiment analysis:
* Total number of positive and negative words
* Difference = Positive words - Negative words
* Percent sentiment = Difference / Total

$ $

In [214]:
positives = open('Data/LoughranMcDonald_Positive.csv', "r").readlines()
positive = [pos.strip().lower().split(',')[0] for pos in positives]
negatives = open('Data/LoughranMcDonald_Negative.csv', "r").readlines()
negative = [neg.strip().lower().split(',')[0] for neg in negatives]

In [215]:
# Stopwords will be removed from the narrative
stop = stopwords.words('english')

In [216]:
# This was used to test whether extracting workd from the narrative works
start_time = time.clock()
c1 = Comp.sample(frac=0.01)

c1['Narr'] = c1['Consumer complaint narrative'].str.lower().str.replace('[^\w\s]',' ')
c1['Clean'] = c1['Narr'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

c1['Pos'] = c1['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word in (positive)]))
c1['Neg'] = c1['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word in (negative)]))

print time.clock() - start_time, "seconds"
display(c1)

8.90954 seconds


,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
155704,xxxxx,Company has responded to the consumer and the ...,"CITIBANK, N.A.",IL,60532,xxxxx,Consent not provided,Web,Closed with monetary relief,Yes,No,2016-05-04,2016-05-04,Credit card or prepaid card,Billing disputes,xxxxx,xxxxx,,
94618,xxxxx,xxxxx,NORTH STATE ACCEPTANCE LLC,NC,283XX,xxxxx,xxxxx,Web,Closed with non-monetary relief,Yes,xxxxx,2017-06-06,2017-06-06,Other financial service,Other,xxxxx,xxxxx,,
645762,The sole income I have at this time is through...,Company chooses not to provide a public response,"BANK OF AMERICA, NATIONAL ASSOCIATION",NY,117XX,xxxxx,Consent provided,Web,Closed with monetary relief,Yes,No,2015-10-07,2015-10-07,Bank account or service - checking account,Deposits and withdrawals,the sole income i have at this time is through...,sole income time wage income direct deposit wi...,,defaulting
500334,xxxxx,xxxxx,Midwest Recovery Systems,MO,631XX,Servicemember,Other,Web,Closed with explanation,No,No,2016-11-15,2016-11-26,Debt collection - other,Cont'd attempts collect debt not owed - Debt i...,xxxxx,xxxxx,,
339590,xxxxx,xxxxx,"CITIBANK, N.A.",MO,63116,Servicemember,xxxxx,Phone,Closed with monetary relief,Yes,Yes,2013-10-23,2013-10-24,Credit card or prepaid card,Late fee,xxxxx,xxxxx,,
604315,xxxxx,Company chooses not to provide a public response,Experian Information Solutions Inc.,TX,77047,xxxxx,Consent not provided,Web,Closed with explanation,Yes,Yes,2015-10-13,2015-10-13,Credit reporting,Credit reporting company's investigation - Pro...,xxxxx,xxxxx,,
569754,xxxxx,xxxxx,"CITIBANK, N.A.",FL,33414,Older American,xxxxx,Phone,Closed with monetary relief,Yes,Yes,2012-08-22,2012-08-23,Credit card or prepaid card,Billing disputes,xxxxx,xxxxx,,
725339,xxxxx,Company has responded to the consumer and the ...,"BANK OF AMERICA, NATIONAL ASSOCIATION",IL,60504,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2016-03-05,2016-03-07,Bank account or service - checking account,Deposits and withdrawals,xxxxx,xxxxx,,
627249,xxxxx,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,FL,32725,xxxxx,xxxxx,Postal mail,Closed with explanation,Yes,No,2017-02-08,2017-02-13,Credit reporting,Incorrect information on credit report - Accou...,xxxxx,xxxxx,,
447869,xxxxx,xxxxx,TD BANK US HOLDING COMPANY,PA,19116,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2013-08-20,2013-08-22,Bank account or service - checking account,Deposits and withdrawals,xxxxx,xxxxx,,


In [217]:
start_time = time.clock()

Comp['Narr'] = Comp['Consumer complaint narrative'].str.lower().str.replace('[^\w\s]',' ')
Comp['Clean'] = Comp['Narr'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

Comp['Pos'] = Comp['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word in (positive)]))
Comp['Neg'] = Comp['Clean'].apply(lambda x: ' '.join([word for word in x.split() if word in (negative)]))

print time.clock() - start_time, "seconds"

923.374355 seconds


In [218]:
Comp.drop(['Consumer complaint narrative'], inplace = True, axis = 1)
display(Comp)

,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
0,xxxxx,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
1,Company has responded to the consumer and the ...,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed
2,xxxxx,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...
3,xxxxx,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,
4,xxxxx,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,
5,xxxxx,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,
6,Company believes it acted appropriately as aut...,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,
7,Company chooses not to provide a public response,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...
8,Company believes it acted appropriately as aut...,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
9,xxxxx,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,


### Company public response

$ $

This feature is largely unpopulated (73%) or gets the standard "no public response" from either the company (6%) or CFPB (14%). This field can be dropped.


In [219]:
Comp['Company public response'].value_counts(normalize = True,dropna = False)

xxxxx                                                                                                                                   0.7297
Company has responded to the consumer and the CFPB and chooses not to provide a public response                                         0.1440
Company chooses not to provide a public response                                                                                        0.0632
Company believes it acted appropriately as authorized by contract or law                                                                0.0452
Company believes the complaint is the result of a misunderstanding                                                                      0.0042
Company disputes the facts presented in the complaint                                                                                   0.0039
Company believes complaint caused principally by actions of third party outside the control or direction of the company                 0.0033

In [220]:
Comp.drop(['Company public response'], inplace = True, axis = 1)
display(Comp)

,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
0,M&T BANK CORPORATION,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
1,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed
2,"CITIZENS FINANCIAL GROUP, INC.",PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...
3,AMERICAN EXPRESS COMPANY,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,
4,"CITIBANK, N.A.",VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,
5,U.S. BANCORP,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,
6,California Accounts Service,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,
7,Experian Information Solutions Inc.,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...
8,"Aldridge Pite, LLP",CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
9,OCWEN LOAN SERVICING LLC,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,


### Companies
$ $

50 companies account for 75% of the complaints even though there are 4300 companies in the dataset. Even among the top companies, the 15 most-complained about companies make up 60% of the complaints.

$ $

In [221]:
Companies = Comp['Company'].value_counts(normalize = True,dropna = False) \
                           .reset_index() \
                           .rename(columns={'index': 'Company', 'Company': 'Perc'})
Companies['Cum_perc'] = Companies['Perc'].cumsum()
      
display(Companies)

,Company,Perc,Cum_perc
0,"BANK OF AMERICA, NATIONAL ASSOCIATION",0.0822,0.0822
1,WELLS FARGO & COMPANY,0.0675,0.1498
2,"EQUIFAX, INC.",0.0661,0.2158
3,Experian Information Solutions Inc.,0.0627,0.2785
4,"TRANSUNION INTERMEDIATE HOLDINGS, INC.",0.0556,0.3341
5,JPMORGAN CHASE & CO.,0.0542,0.3884
6,"CITIBANK, N.A.",0.0440,0.4324
7,OCWEN LOAN SERVICING LLC,0.0300,0.4624
8,CAPITAL ONE FINANCIAL CORPORATION,0.0261,0.4885
9,"Navient Solutions, LLC.",0.0241,0.5126


In [222]:
Comp.drop(['Company'], inplace = True, axis = 1)
display(Comp)

,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
0,MI,48382,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
1,AL,352XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed
2,PA,177XX,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...
3,ID,83854,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,
4,VA,23233,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,
5,MN,48322,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,
6,CA,92111,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,
7,VA,224XX,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...
8,CA,93101,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
9,FL,32714,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,


### State and ZIP
$ $
Complaints distrubution is also skewed heavily towards states with larger population. State and ZIP code features are dropped from the analysis.
$ $


In [223]:
States = Comp['State'].value_counts(normalize = True,dropna = False) \
                      .reset_index() \
                      .rename(columns={'index': 'State', 'State': 'Perc'})
States['Cum_perc'] = States['Perc'].cumsum()
      
display(States)

,State,Perc,Cum_perc
0,CA,0.1418,0.1418
1,FL,0.0957,0.2375
2,TX,0.0785,0.3160
3,NY,0.0678,0.3839
4,GA,0.0483,0.4321
5,NJ,0.0391,0.4712
6,IL,0.0367,0.5079
7,PA,0.0353,0.5432
8,VA,0.0314,0.5746
9,MD,0.0309,0.6056


In [224]:
Comp.drop(['State', 'ZIP code'], inplace = True, axis = 1)
display(Comp)

,Tags,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
0,xxxxx,xxxxx,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
1,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed
2,Older American,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...
3,Older American,xxxxx,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,
4,xxxxx,xxxxx,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,
5,xxxxx,xxxxx,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,
6,xxxxx,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,
7,xxxxx,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...
8,xxxxx,xxxxx,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
9,Older American,xxxxx,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,


### Tags

In [225]:
Comp['Tags'].value_counts(normalize = True,dropna = False)

xxxxx                                         0.8599
Older American                                0.0767
Servicemember                                 0.0522
Older American, Servicemember                 0.0112
Name: Tags, dtype: float64

In [226]:
Comp.drop(['Tags'], inplace = True, axis = 1)

### Consumer consent provided?

In [227]:
Comp['Consumer consent provided?'].value_counts(normalize = True,dropna = False)

xxxxx                                0.6106
Consent provided                     0.2101
Consent not provided                 0.1663
Other                                0.0127
Consent withdrawn                    0.0003
Name: Consumer consent provided?, dtype: float64

In [228]:
Comp['Consumer consent provided?']\
    .replace(['xxxxx','Consent withdrawn'],['Other','Other'],inplace=True)
      
Comp['Consumer consent provided?'].value_counts(normalize = True,dropna = False)

Other                                0.6236
Consent provided                     0.2101
Consent not provided                 0.1663
Name: Consumer consent provided?, dtype: float64

In [229]:
display(Comp)

,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg
0,Other,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
1,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed
2,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...
3,Other,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,
4,Other,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,
5,Other,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,
6,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,
7,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...
8,Other,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,
9,Other,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,


In [230]:
Comp['Submitted via'].value_counts(normalize = True,dropna = False)

Web                         0.6906
Referral                    0.1655
Phone                       0.0663
Postal mail                 0.0634
Fax                         0.0137
Email                       0.0004
Name: Submitted via, dtype: float64

In [231]:
Comp['Company response to consumer'].value_counts(normalize = True,dropna = False)

Closed with explanation                         0.7529
Closed with non-monetary relief                 0.1223
Closed with monetary relief                     0.0651
Closed without relief                           0.0215
Closed                                          0.0212
Closed with relief                              0.0064
In progress                                     0.0058
Untimely response                               0.0048
Name: Company response to consumer, dtype: float64

In [232]:
Comp['Timely response?'].value_counts(normalize = True,dropna = False)

Yes                 0.9719
No                  0.0281
Name: Timely response?, dtype: float64

In [233]:
# For about 7.5% of observations, the answer is blank. recode as Other
Comp['Consumer disputed?']\
    .replace(['xxxxx'],['Other'],inplace=True)
Comp['Consumer disputed?'].value_counts(normalize = True,dropna = False)

No                    0.7468
Yes                   0.1787
Other                 0.0745
Name: Consumer disputed?, dtype: float64

In [234]:
Comp['DTDiff'] = (Comp['DTSent'] - Comp['DTRec']).dt.days
Comp['Yr'] = Comp['DTRec'].dt.year
Comp['Mon'] = Comp['DTRec'].dt.month

In [235]:
Comp['DTDiff'].value_counts(normalize = True,dropna = False)

 0                    0.5023
 1                    0.1109
 2                    0.0731
 3                    0.0575
 4                    0.0536
 5                    0.0476
 6                    0.0345
 7                    0.0217
 8                    0.0091
-1                    0.0085
 9                    0.0053
 10                   0.0045
 13                   0.0039
 11                   0.0039
 14                   0.0038
 12                   0.0037
 15                   0.0034
 20                   0.0028
 21                   0.0026
 16                   0.0025
 17                   0.0025
 19                   0.0025
 18                   0.0022
 22                   0.0017
 34                   0.0015
 35                   0.0015
 28                   0.0014
 27                   0.0013
 26                   0.0013
 33                   0.0012
 25                   0.0012
 23                   0.0012
 36                   0.0011
 29                   0.0011
 32           

In [236]:
# Not surprisingly, how a complaint was submitted  
# has an effect on how long it takes CFPB to submit it
Comp.groupby('Submitted via')['DTDiff'].mean()

Submitted via
Email                       8.2241
Fax                         6.5495
Phone                       4.7141
Postal mail                 8.4231
Referral                    5.9806
Web                         3.0818
Name: DTDiff, dtype: float64

In [237]:
# Fold the extreme values of DTDiff
#Comp['DTDiff'].clip(0, 8): doesn't do "in place" replacement

Comp['DTDiff'][Comp['DTDiff'] > 8] = 8
Comp['DTDiff'][Comp['DTDiff'] < 0] = 0

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [238]:
Comp['DTDiff'].value_counts(normalize = True,dropna = False)

0                 0.5108
1                 0.1109
8                 0.0904
2                 0.0731
3                 0.0575
4                 0.0536
5                 0.0476
6                 0.0345
7                 0.0217
Name: DTDiff, dtype: float64

In [239]:
Comp['Yr'].value_counts(normalize = True,dropna = False)

2016                 0.2306
2015                 0.2029
2014                 0.1843
2017                 0.1616
2013                 0.1303
2012                 0.0872
2011                 0.0031
Name: Yr, dtype: float64

In [240]:
Comp['Mon'].value_counts(normalize = True,dropna = False)

3                  0.0962
7                  0.0960
6                  0.0932
5                  0.0920
4                  0.0915
1                  0.0880
2                  0.0841
8                  0.0771
10                 0.0742
9                  0.0717
12                 0.0700
11                 0.0660
Name: Mon, dtype: float64

In [241]:
display(Comp)

,Consumer consent provided?,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,DTRec,DTSent,Prod,Iss,Narr,Clean,Pos,Neg,DTDiff,Yr,Mon
0,Other,Referral,Closed with explanation,Yes,No,2014-03-12,2014-03-17,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,,5,2014,3
1,Consent provided,Web,Closed with explanation,Yes,No,2016-10-01,2016-10-05,Credit reporting,Incorrect information on credit report - Accou...,i have outdated information on my credit repor...,outdated information credit report previously ...,,outdated disputed,4,2016,10
2,Consent provided,Web,Closed with explanation,Yes,No,2016-10-17,2016-10-20,Consumer loan - vehicle loan,Managing the loan or lease,i purchased a new car on xxxx xxxx the car de...,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...,3,2016,10
3,Other,Web,Closed with explanation,Yes,Yes,2014-06-08,2014-06-10,Credit card or prepaid card,Other,xxxxx,xxxxx,,,2,2014,6
4,Other,Web,Closed with explanation,Yes,Yes,2014-09-13,2014-09-13,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,xxxxx,,,0,2014,9
5,Other,Phone,Closed with monetary relief,Yes,No,2013-11-13,2013-11-20,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,xxxxx,,,7,2013,11
6,Consent not provided,Web,Closed with explanation,Yes,No,2015-06-16,2015-06-19,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,xxxxx,,,3,2015,6
7,Consent provided,Web,Closed with explanation,Yes,No,2015-06-15,2015-06-15,Credit reporting,Credit reporting company's investigation - Other,an account on my credit report has a mistaken ...,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...,0,2015,6
8,Other,Referral,Closed with explanation,Yes,Yes,2015-11-13,2015-12-10,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,,8,2015,11
9,Other,Web,Closed with explanation,Yes,No,2014-10-21,2014-10-21,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,xxxxx,,,0,2014,10


In [242]:
list(Comp)

['Consumer consent provided?',
 'Submitted via',
 'Company response to consumer',
 'Timely response?',
 'Consumer disputed?',
 'DTRec',
 'DTSent',
 'Prod',
 'Iss',
 'Narr',
 'Clean',
 'Pos',
 'Neg',
 'DTDiff',
 'Yr',
 'Mon']

In [243]:
CompFinal=Comp.drop(['DTRec','DTSent','Narr'], axis = 1)\
              .rename(columns = {'Consumer consent provided?':'Consent',\
                                 'Submitted via':'Via',\
                                 'Company response to consumer':'Response',\
                                 'Timely response?':'Timely',\
                                 'Consumer disputed?':'Disputed'})

In [244]:
display(CompFinal)
CompFinal.to_csv("CompFinal.csv", index = False, quoting=csv.QUOTE_NONNUMERIC)
print time.clock() - beg_time, "seconds"

,Consent,Via,Response,Timely,Disputed,Prod,Iss,Clean,Pos,Neg,DTDiff,Yr,Mon
0,Other,Referral,Closed with explanation,Yes,No,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,,,5,2014,3
1,Consent provided,Web,Closed with explanation,Yes,No,Credit reporting,Incorrect information on credit report - Accou...,outdated information credit report previously ...,,outdated disputed,4,2016,10
2,Consent provided,Web,Closed with explanation,Yes,No,Consumer loan - vehicle loan,Managing the loan or lease,purchased new car xxxx xxxx car dealer called ...,good satisfied,overpaid questioned delay deliberately delayin...,3,2016,10
3,Other,Web,Closed with explanation,Yes,Yes,Credit card or prepaid card,Other,xxxxx,,,2,2014,6
4,Other,Web,Closed with explanation,Yes,Yes,Debt collection - credit card,Communication tactics - Frequent or repeated c...,xxxxx,,,0,2014,9
5,Other,Phone,Closed with monetary relief,Yes,No,Mortgage - conventional adjustable mortgage (ARM),"Loan servicing, payments, escrow account",xxxxx,,,7,2013,11
6,Consent not provided,Web,Closed with explanation,Yes,No,Debt collection - medical,Improper contact or sharing of info - Other,xxxxx,,,3,2015,6
7,Consent provided,Web,Closed with explanation,Yes,No,Credit reporting,Credit reporting company's investigation - Other,account credit report mistaken date mailed deb...,incredibly satisfied,mistaken suspicious incapable offended suspici...,0,2015,6
8,Other,Referral,Closed with explanation,Yes,Yes,Mortgage - other,"Loan modification,collection,foreclosure",xxxxx,,,8,2015,11
9,Other,Web,Closed with explanation,Yes,No,Mortgage - conventional fixed mortgage,"Loan modification,collection,foreclosure",xxxxx,,,0,2014,10


1259.482984 seconds


#### Unused code

def Clean_Narr(row):
    text = row['Consumer complaint narrative'].lower()
    if text == 'xxxxx':
        text = '  '
    text = RegexpTokenizer(r'\w+').tokenize(text)
    clean_text = filter(lambda x: x not in stopwords.words('english'), text)
    clean_text = ' '.join(clean_text).strip()
    return clean_text

def Count_neg(row):
    text = row['Narr'].split()
    neg = len([word for word in text if word in negative])
    return neg

def Count_pos(row):
    text = row['Narr'].split()
    pos = len([word for word in text if word in positive])
    return pos

Comp['Tot'] = Comp['Narr'].str.split().str.len()
Comp['Neg'] = Comp.apply(Count_neg,axis=1)
Comp['Pos'] = Comp.apply(Count_pos,axis=1)

start_time = timeit.default_timer()
Comp['Narr'] = Comp.apply(Clean_Narr,axis=1)
Comp['Tot'] = Comp['Narr'].str.split().str.len()
Comp['Neg'] = Comp.apply(Count_neg,axis=1)
Comp['Pos'] = Comp.apply(Count_pos,axis=1)
print(timeit.default_timer() - start_time)

start_time = timeit.default_timer()
Comp['Narr'] = Comp.apply(Clean_Narr,axis=1)
Comp['Tot'] = Comp['Narr'].str.split().str.len()
Comp['Neg'] = Comp.apply(Count_neg,axis=1)
Comp['Pos'] = Comp.apply(Count_pos,axis=1)
print(timeit.default_timer() - start_time)
